### Prepare raw data

Download html files from site

In [1]:
#!wget -r -k -l 7 -p -E -nc https://povar.ru/ratingCountratingCount
!rm -rf ./chroma_db

### Make DataSet

In [2]:
from utils import make_data_set, read_recipes_pkl, save_recipes_pkl, make_recipes,filter_recipe
from Recipe import Recipe, RecipesProject

MAIN_DIR = "/home/zaderu/Documents/MLHS/LLM Project/data/povar.ru/recipes/"
FILE_NAME = "/workspace/llm_project/raw_recipes/povar_recipes.pickle"

/workspace/Recipes/rec_proj_venv/lib/python3.10/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# povar_recipes = make_data_set(MAIN_DIR)
# save_recipes_pkl(povar_recipes, FILE_NAME)

# povar_recipes = read_recipes_pkl(FILE_NAME)
# recipes_list = make_recipes(povar_recipes, output_file="./Recipe_recipes.pickle")
# recipes_list = filter_recipe(recipes_list, max_steps = 10, max_min=120, min_rating=4.3, min_votes=5)
# save_recipes_pkl(recipes_list, "./povar_recipes_final.pickle")

In [4]:
recipes_list = read_recipes_pkl("./povar_recipes_final.pickle")
len(recipes_list)

Collected 16052 recipes


16052

### Create Chroma DB

In [5]:
# Chroma DB

from build_chroma_db import build_chroma_db, requst_chroma_db
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
vector_store, tokenizer = build_chroma_db(
    recipes_list,
    embedding_model="ai-forever/sbert_large_nlu_ru", #"nomic-ai/nomic-embed-text-v1.5", #"IlyaGusev/saiga_yandexgpt_8b" 
    device=device,
)

38267 chunks are to be loaded to chroma db


In [6]:
len(vector_store.get()['ids'])

38267

### Create Knowledgr Graph

In [7]:
### Knowledge graph

from build_knowledge_graph import make_tags_list, lemmatize_tags, build_knowledge_graph\
    , lemmatize, lemmatize_sentance, make_one_word_tags_list, enreach_query_with_relative_tags

tags = make_tags_list(recipes_list)

lemmatize_tags(tags)

one_word_tags = make_one_word_tags_list(tags)

In [8]:
G = build_knowledge_graph(recipes_list, tags)

In [9]:
rp = RecipesProject(
    recipes=recipes_list,
    knowledgeGraph=G,
    tags=tags,
    vectorStore=vector_store,
    oneWordTags=one_word_tags
)
# rp.add_resipes_list(recipes_list)
# rp.add_knowledge_graph(G)
# rp.add_tags(tags)
# rp.add_one_word_tags(one_word_tags)

In [10]:
max_len = 0
for tag in tags:
    if len(tags[tag]['lemma']) > max_len:
        max_len = len(tags[tag]['lemma'])
max_len # longest tag

4

### Add names to Charoma DB

In [11]:
# from langchain_core.documents import Document
# import uuid

# uuid = [str(uuid.uuid4()) for r in rp.recipes]
# docs = [Document(page_content=r.name, metadata={'uuid': r.uuid}) for r in rp.recipes]
# vector_store.add_documents(documents=docs, ids=uuid)
# rp.recipes[0].document, docs[0]

In [13]:
len(vector_store.get()['ids'])

38267

### Create LLM

In [14]:
from build_ollama import build_model, llm_invoke
llm = build_model()

### Tunning

In [33]:
requests = [
    "Какой рецепт вегетарианской пасты?",
    "Как приготовить блинчики?",
    "Что можно приготовить из куриного филе?",
    "Как сделать шоколадный торт?",
    "Как приготовить рис для суши?",
    "Как приготовить суп-пюре из тыквы?",
    "Что можно приготовить из картошки и сыра?",
    "Как сделать домашний хлеб?",
    "Какие десерты можно приготовить без муки?",
    "Как приготовить рис с курицей и овощами?",
    "Как сделать соус для пасты?",
    "Как приготовить морковные котлеты?",
    "Что приготовить на ужин за 30 минут?",
    "Как сделать домашнюю пиццу?",
    "Что можно приготовить из куриного филе, риса и брокколи?",
    "Как приготовить сладкий омлет?",
    "Как сделать домашнее мороженое без мороженицы?",
    "Что можно приготовить для обеда из мяса и риса?",
    "Как приготовить крем-суп из грибов?",
    "Как приготовить рыбу с картошкой в духовке?",
    "Что приготовить для завтрака, если есть яйца, авокадо и помидоры?",
    "Как приготовить куриные крылышки на гриле?",
    "Что приготовить на праздничный ужин?",
    "Как приготовить соус бешамель?",
    "Как сделать торт без яиц?"
]

In [18]:
query = "Как сделать балоньезе"

In [21]:
answer_kg = rp.invoke(query, verbose=True)
if len(answer_kg[0]) == 0:
    answer_kg = answer_kg[1]
else:
    answer_kg = answer_kg[0]
print(len(answer_kg))

New tags are not added
16052


### Queries

In [36]:
recipes_dict = {r.uuid: r.document.page_content for r in rp.recipes}

In [44]:
import random
def final_query(query):
    query = query.lower().replace("рецепт","")
    answer_kg = rp.invoke(query, verbose=False)
    if len(answer_kg[0]) == 0:
        answer_kg = answer_kg[1]
    else:
        answer_kg = answer_kg[0]

    results_db = requst_chroma_db(vector_store, tokenizer, query, device, k=50)
    

    res_final = None
    if len(answer_kg) < 1000:
        uuid_kg = [uuid for uuid in answer_kg]
        for r in results_db:
            if r.metadata['uuid'] in uuid_kg:
                res_final = r.metadata['uuid']
                return recipes_dict[res_final], "KG+DB"
        return recipes_dict[random.sample(answer_kg, 1)[0]], "KG" 


    answer_llm = llm_invoke(llm, query)
    results_db = requst_chroma_db(vector_store, tokenizer, answer_llm, device, k=1)

    return results_db, "LLM+DB"

In [45]:
for query in requests:
    res, type = final_query(query)
    print(f"{query}: {type}\n{res}\n\n")
    

/tmp/ipykernel_6077/2232769856.py:20: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  return recipes_dict[random.sample(answer_kg, 1)[0]], "KG"


Какой рецепт вегетарианской пасты?: KG
Ужин без мяса Как приготовить ужин без мяса? Я задалась этим вопросом, когда приехали к нам в гости друзья-вегетарианцы. И чем же накормить трёх мужиков сытно и вкусно, если не мясом? У нас была паста с овощами. Подавайте макароны горячими, только со сковородки, сверху можете присыпать тертым пармезаном. Рецепт разработан таким образом, что он не теряет вкуса и аромата без мяса. Грибы и помидоры - старый добрый прием для обычных макарон. Все наши гости остались довольны. Ужин без мяса - это не только полезно, но и вкусно.. Количество порций: 4.0. Ингридиенты паста: 500 грамм. лук: 1 штука. чеснок: 2 зубчика. сладкий перец: 1 штука. грибы шампиньоны: 200 грамм. помидоры: 2 штуки. сахар: 2 ст. ложки. соль, перец: по вкусу. сухие итальянские травы: по вкусу. Способ приготовления 1. На разогретой сковороде с маслом жарьте измельченный лук и чеснок, пока лук не станет прозрачным. А в отдельной кастрюле отварите макароны до полуготовности.. 2. К луку от

/tmp/ipykernel_6077/2232769856.py:20: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  return recipes_dict[random.sample(answer_kg, 1)[0]], "KG"


Что приготовить на ужин за 30 минут?: KG
Плов с нутом в мультиварке Желаете отведать вкусного вегетарианского плова? Приготовьте плов с нутом и рисом в вашей мультиварке! Благодаря специям, плов получается очень ароматным, а изюм и фисташки станут изюминкой блюда! Как приготовить плов с нутом в мультиварке в домашних условиях? С помощью мультиварки это очень просто! Единственное, хотелось бы напомнить, что нут нужно предварительно замочить на ночь в чистой воде. От количества времени, проведенного в воде, напрямую зависит мягкость нута. Удачного плова!. Количество порций: 2.0. Ингридиенты вода: 320 мл. изюм: 20 грамм. лук репчатый: 1 штука. морковь: 1 штука. рис "басмати": 120 грамм. тыква: 120 грамм. чеснок: 1 штука. зира: 1 ч. ложка. карри: 1 ч. ложка. масло оливковое: 3 ст. ложки. нут: 100 грамм. фисташки: 20 грамм. соль: 1 ч. ложка. Способ приготовления 1. Предварительно замоченный на ночь нут отвариваем на медленном огне 40 минут.. 2. Рис замочите в теплой воде.. 3. Морковь, тыкву

/tmp/ipykernel_6077/2232769856.py:20: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  return recipes_dict[random.sample(answer_kg, 1)[0]], "KG"


Что можно приготовить из куриного филе, риса и брокколи?: KG
Запеченное куриное филе с брокколи Запеченное куриное филе с брокколи - потрясающий вариант горячего блюда! Полуфабрикаты для него можно приготовить заранее, например, накануне, а само нежное, сочное блюдо готовится примерно 20 минут. Приготовим запеченное куриное филе с брокколи в домашних условиях. Для начала необходимо сварить куриное филе, рис и брокколи. Куриное филе нарезать небольшими кусочками, а сыр следует натереть на средней терке. В отдельной миске смешать все ингредиенты. Выпекать следует в течение 20-25 минут при температуре 180 градусов. Подавать к столу горячим.. Количество порций: 4.0. Ингридиенты куриное филе: 400-500 грамм. брокколи: 400 грамм. рис: 1 стакан. сливки 10-25%: 200 мл. сыр тертый: 100-150 грамм. соль: 1 щепотка. перец: 1 щепотка. Способ приготовления 1. Итак, ставим на огонь в кастрюле с водой куриное филе, доводим до кипения 2 стакана воды для риса и еще одну кастрюлю с водой для брокколи. Кур

/tmp/ipykernel_6077/2232769856.py:20: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  return recipes_dict[random.sample(answer_kg, 1)[0]], "KG"


Как приготовить соус бешамель?: KG
Карамель "Петушок" Вкус этого леденца знаком с детства. Хотите освежить свои воспоминания и побаловать близких натуральными сладостями? Тогда рецепт приготовления карамели "Петушок" — для вас. При наличии специальной формы приготовить карамель "Петушок" в домашних условиях может даже ребенок. При желании рецепт можно дополнить пищевыми красителями, сделав леденцы разноцветными.. Количество порций: 6.0. Ингридиенты сахар: 8 ст. ложек. вода: 1 ст. ложка. уксус яблочный: 1 ст. ложка. Способ приготовления 1. В небольшую кастрюльку или сотейник всыпьте сахар.. 2. Добавьте чистую воду комнатной температуры.. 3. Добавить 6% уксус.. 4. Тщательно перемешайте все еще до того, как отправите вариться.. 5. Отправьте на огонь. Нужно постоянно помешивать, чтобы сахар равномерно растворялся. В рецепт приготовления карамели "Петушок" можно добавить любой пищевой краситель или ароматизатор.. 6. Варить карамель нужно на сильном огне. Тут есть один нюанс: чем дольше она 

In [ ]:
print(answer_llm)

In [ ]:
answer_kg = rp.invoke(query, verbose=True)
if len(answer_kg[0]) == 0:
    answer_kg = answer_kg[1]
else:
    answer_kg = answer_kg[0]

In [24]:
query

'Как сделать балоньезе'

In [ ]:
answer_llm, query

In [26]:
vector_store.similarity_search(query, k=50)

[Document(id='bff8cb0e-929e-4359-8e14-a7e623160df6', metadata={'name': 'Мороженое "Шарики"', 'uuid': '7b2deb8b-97fd-4c08-b817-75ad040162f3'}, page_content='Разогреть очень сильно большое количество растительного масла без запаха. Выложить туда шарики и обжарить до золотистой корочки буквально полминуты. Быстро достать и выложить на салфетку. Мороженое "Шарики" в домашних условиях можно подавать к столу.. Калории: 658 ккал. Белки: 3 г. Жиры: 63 г. Углеводы: 20 г. Время: 1 ч.. Тип блюда: Десерты, Мороженое. Назначение: на ужин, для детей, на десерт, на праздничный стол, детский. Диета: None. Основные ингредиенты: молочные продукты. География кухни: None. Средняя оценка: 4.4. Количество оценок: 7.0.'),
 Document(id='a0d6d358-a986-43d8-bdf6-a6aeb8d57007', metadata={'name': 'Пунш Аэлин', 'uuid': 'd98d9e8d-2112-42b9-943d-88e84cfa3786'}, page_content='заменить на свежевыжатый сок лайма, его понадобится 1 стакан.. 3. В чаше для пунша смешать соки и концентрат лайма (или же свежевыжатый сок).. 

In [ ]:
results_db = requst_chroma_db(vector_store, tokenizer, query, device, k=5)

In [ ]:
[d.metadata['name'] for d in results_db]

In [ ]:
result = set([d.metadata['uuid'] for d in results_db]).intersection(set([uuid for uuid in answer_kg[1]]))

In [ ]:
[for r in result]

In [ ]:
[r.name for r in rp.recipes if r.name == ""]

In [ ]:
[
    ['Как приготовить карбонару?','Что нужно для соуса?'],
    ['Что приготовить детского с мясом?'],
    ['Быстрый рецепт пирога из яблок?'],
    ['Что сделать из кабачков?'],
    ['У меня есть картошка, морковь и духовка. Что мне можно сделать?'],
    ['Дай мне рецепт жареной рыбы с картофелем.'],
    ['Мои дети голодные, а у меня нет времени. Как их накормить?'],
    ['Что нужно для фруктового салата со сладкими фруктами.','Какие фрукты можно использовать вместо дыни?']
]